<a href="https://colab.research.google.com/github/melmar-g1thub/INTERPOLATING-NEURAL-NETWORK/blob/main/preparation_of_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PREPOCESSING OF DATA
- Upload all the necesary libraries used throughout the modules
- Upload the data files and create the indexed mapping
- Preparte data for NN training:
1. Splitting into train/validation/test sets for finding parameters, hyperparameters and generalization
2. Normalization
3. Convert into nn.module Tensors

### UPLOADING LIBRARIES

In [ ]:
!pip install skorch
from skorch import NeuralNetRegressor
from skorch.callbacks import EarlyStopping

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import seaborn as sns
import random
import itertools
import json
import joblib
import time
import ast
import csv
import os

from sklearn.model_selection import train_test_split, RandomizedSearchCV, RepeatedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, QuantileTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer
from sklearn.ensemble import RandomForestRegressor
from matplotlib.ticker import ScalarFormatter

from datetime import datetime
from tqdm import tqdm
from scipy.stats import randint, loguniform
from scipy.interpolate import griddata, SmoothBivariateSpline

try:
    from torch.amp import autocast
except ImportError:
    from torch.cuda.amp import autocast

try:
    from torch.amp import GradScaler
except ImportError:
    from torch.cuda.amp import GradScaler

### UPLOAD DATA

In [ ]:
# Load the data
import os
if not os.path.exists("/content/neutron-star-eos"):
    !git clone https://github.com/tu-usuario/neutron-star-eos.git
    data_path = "/content/neutron-star-eos/data/"

# temperature and baryon number data
eos_t = np.loadtxt(data_path + 'eos.t', skiprows=2)  # 81 values
eos_nb = np.loadtxt(data_path + 'eos.nb', skiprows=2)  # 326 values
eos_yq = np.zeros(len(eos_t))  # Constant value of 0

# thermodynamic data
thermo_data = np.loadtxt(data_path + 'eos.thermo', skiprows=1)

In [ ]:
# Extract indices and thermodynamic properties
indices = thermo_data[:, :3].astype(int)  # i_T, i_nb, i_Yq
q_values = thermo_data[:, 3:5]  # Q1 (pressure) and Q2 (entropy)

# Convert to zero-based index
i_t = indices[:, 0] - 1
i_nb = indices[:, 1] - 1

# Ensure indices are valid, np.where() to get valid integer and not boolean
valid_indices = np.where((i_t >= 0) & (i_t < len(eos_t)) & (i_nb >= 0) & (i_nb < len(eos_nb)))[0]
T_valid = eos_t[i_t[valid_indices]]
nb_valid = eos_nb[i_nb[valid_indices]]
q_valid = q_values[valid_indices]

logT = np.log10(T_valid)
log_nb = np.log10(nb_valid)

P_valid = q_valid[:, 0] * nb_valid
S_valid = q_valid[:, 1] * nb_valid

logP = np.log10(np.clip(P_valid, 1e-10, None))  # protects invalid log10(0)
logS = np.log10(np.clip(S_valid, 1e-10, None))

In [ ]:
# Filter central 90%
T_q05, T_q95 = np.quantile(logT, [0.05, 0.95])
nb_q05, nb_q95 = np.quantile(log_nb, [0.05, 0.95])
mask = (logT >= T_q05) & (logT <= T_q95) & \
       (log_nb >= nb_q05) & (log_nb <= nb_q95)

T_values = logT[mask]
nb_values = log_nb[mask]
P_values = logP[mask]
S_values = logS[mask]

inputs = np.vstack([T_values, nb_values]).T
outputs = np.vstack([P_values, S_values]).T

print("Initial inputs shape (before splitting/scaling):", inputs.shape)
print("Initial outputs shape (before splitting/scaling):", outputs.shape)

In [ ]:
# Histograms for individual features
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
sns.histplot(inputs[:, 0], kde=True, bins=50, color='skyblue')
plt.title('Distribution of log10(Temperature)', fontsize=18)
plt.xlabel('log10(T)', fontsize=17)
plt.ylabel('Count', fontsize=17)
plt.tick_params(axis='both', labelsize=14)

plt.subplot(2, 2, 2)
sns.histplot(inputs[:, 1], kde=True, bins=50, color='lightcoral')
plt.title('Distribution of log10(Baryon Number)', fontsize=18)
plt.xlabel('log10(nb)', fontsize=17)
plt.ylabel('Count', fontsize=17)
plt.tick_params(axis='both', labelsize=14)

plt.subplot(2, 2, 3)
sns.histplot(outputs[:, 0], kde=True, bins=50, color='lightgreen')
plt.title('Distribution of log10(Pressure)', fontsize=18)
plt.xlabel('log10(P)', fontsize=17)
plt.ylabel('Count', fontsize=17)
plt.tick_params(axis='both', labelsize=14)

plt.subplot(2, 2, 4)
sns.histplot(outputs[:, 1], kde=True, bins=50, color='orchid')
plt.title('Distribution of log10(Entropy)', fontsize=18)
plt.xlabel('log10(S)', fontsize=17)
plt.ylabel('Count', fontsize=17)
plt.tick_params(axis='both', labelsize=14)

plt.subplots_adjust(wspace=0.5, hspace=0.8)
plt.tight_layout()
plt.suptitle('Distributions of Log-Transformed EoS Features (After 90% Filter)', y=1.02, fontsize=16)
plt.show()

In [ ]:
# 2D Histogram / Density Plot for Input Parameters (T vs nb)
plt.figure(figsize=(10, 8))
plt.hist2d(inputs[:, 0], inputs[:, 1], bins=50, cmap='viridis')
plt.colorbar(label='Count')
plt.xlabel('log10(T)')
plt.ylabel('log10(nb)')
plt.title('2D Distribution of Input Parameters (T vs nb)')
plt.grid(True, linestyle='--', alpha=0.6)
plt.savefig(os.path.join(plot_save_path, '2d_input_distribution.png'), dpi=300, bbox_inches='tight')
plt.show()

plt.figure(figsize=(10, 8))
sns.kdeplot(x=inputs[:, 0], y=inputs[:, 1], fill=True, cmap='viridis', levels=20)
plt.xlabel('log10(T)')
plt.ylabel('log10(nb)')
plt.title('2D Density Plot of Input Parameters (T vs nb)')
plt.savefig(os.path.join(plot_save_path, '2d_density_plot.png'), dpi=300, bbox_inches='tight')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

From the histograms we check:
- Inputs (T and nb) are uniformly distributed, showing approximatly the same number of counts across their range
- Outputs are slightly skewed, S specifically shows a less gaussian behaviour

Stratification of inputs will not be required as after splitting we'll still get pretty uniform distributions for each set.
We will look carefully into how splitting and normalization affect to the outputs of the 3 data sets, as we hope them all to be representative of the behaviour of the EoS.


### PREPARATION OF DATA

Normalization of the files needs to be performed AFTER it's been divided in the train-validation-test sets!!!!!!!
Because if we perform it before it causes:
- Global Statistics: normalization process calculates these statistics (mean, std, min, max) using all the data points, including those in what will eventually become your test set.
- Unfair Advantage: training set's scaled values are influenced by the statistics of the test set. This means your model indirectly "sees" information about the test data during training.

In [ ]:
# 1. Split the data
# First split: 80% train, 20% temp (for validation/test)
in_train, in_temp, out_train, out_temp = train_test_split(
    inputs, outputs, test_size=0.20, random_state=42, shuffle=True)

# Second split: From the 20% temp, split it into 15% validation and 5% test
in_val, in_test, out_val, out_test = train_test_split(
    in_temp, out_temp, test_size=0.25, random_state=42, shuffle=True)

print(f"Train samples: {len(in_train)}")
print(f"Validation samples: {len(in_val)}")
print(f"Test samples: {len(in_test)}")

Fit the Scaler ONLY on the Training Data, this calculates the necessary scaling parameters (mean, std, min, max) exclusively from the training distribution.

Then, we apply transform() to the test and validation data, using the same fitted scaler.

In [ ]:
# 2. Normalization
from sklearn.preprocessing import RobustScaler
scaler_T = StandardScaler()
scaler_nb = StandardScaler()
scaler_P =  QuantileTransformer(output_distribution='normal', n_quantiles=1000, subsample=1_000_000, random_state=42)
scaler_S = QuantileTransformer(output_distribution='normal', n_quantiles=1000, subsample=1_000_000, random_state=42)

# Save the scalers in case we later re-study the model
scaler_save_path = "/content/drive/My Drive/Colab Notebooks/TFG_NN_FINAL/SCALERS"
scaler_P = joblib.load(os.path.join(scaler_save_path, "scaler_P.pkl"))
scaler_S = joblib.load(os.path.join(scaler_save_path, "scaler_S.pkl"))
scaler_T = joblib.load(os.path.join(scaler_save_path, "scaler_T.pkl"))
scaler_nb = joblib.load(os.path.join(scaler_save_path, "scaler_nb.pkl"))

# StandardScaler:	Gaussian-like distribution, improves convergence with activation functions
# MinMaxScaler: Best when we want normalized plots or comparisons, may want to clip/clamp outputs easily and care about scaled MAE/MSE
# QuantileTransformer: For highly skewed distributions, normalize close to gaussian

# We fit the scaler ONLY ON TRAINING SET
def fit_train_scaler(tensor, scaler):
  return scaler.fit_transform(tensor.reshape(-1, 1))

# Then we simply apply the transformation to validation and test set
def apply_scaler(tensor, scaler):
  return scaler.transform(tensor.reshape(-1, 1))

### INPUTS ###
in_T_train_scaled = fit_train_scaler(in_train[:, 0], scaler_T)
in_nb_train_scaled = fit_train_scaler(in_train[:, 1], scaler_nb)

in_T_val_scaled = apply_scaler(in_val[:, 0], scaler_T)
in_nb_val_scaled = apply_scaler(in_val[:, 1], scaler_nb)
in_T_test_scaled = apply_scaler(in_test[:, 0], scaler_T)
in_nb_test_scaled = apply_scaler(in_test[:, 1], scaler_nb)

# Recombine inputs for each set
in_train_processed = np.hstack((in_T_train_scaled, in_nb_train_scaled))
in_val_processed = np.hstack((in_T_val_scaled, in_nb_val_scaled))
in_test_processed = np.hstack((in_T_test_scaled, in_nb_test_scaled))


### OUTPUTS ###
out_P_train_scaled = fit_train_scaler(out_train[:, 0], scaler_P)
out_S_train_scaled = fit_train_scaler(out_train[:, 1], scaler_S)

out_P_val_scaled = apply_scaler(out_val[:, 0], scaler_P)
out_S_val_scaled = apply_scaler(out_val[:, 1], scaler_S)
out_P_test_scaled = apply_scaler(out_test[:, 0], scaler_P)
out_S_test_scaled = apply_scaler(out_test[:, 1], scaler_S)

out_train_processed = np.hstack((out_P_train_scaled, out_S_train_scaled))
out_val_processed = np.hstack((out_P_val_scaled, out_S_val_scaled))
out_test_processed = np.hstack((out_P_test_scaled, out_S_test_scaled))


# Check the shapes of the input and output arrays
print("Training: inputs shape:", in_train_processed.shape)  # (number_of_samples, 2)
print("Training: Outputs shape:", out_train_processed.shape)

# If we were to upload the scalers again
# scaler_save_path = "/content/drive/My Drive/Colab Notebooks/TFG_NN_FINAL/SCALERS"
# joblib.dump(scaler_P, os.path.join(scaler_save_path, "scaler_P.pkl"))
# joblib.dump(scaler_S, os.path.join(scaler_save_path, "scaler_S.pkl"))
# joblib.dump(scaler_T, os.path.join(scaler_save_path, "scaler_T.pkl"))
# joblib.dump(scaler_nb, os.path.join(scaler_save_path, "scaler_nb.pkl"))

In [ ]:
# Visualize normalization for Input Features (T and nb) and Output (logP and logS) features
def visualize_normalization(original_input, normalized_input, original_output, normalized_output, dataset_name, save_path):
    os.makedirs(save_path, exist_ok=True)

    # INPUTS
    fig, axs = plt.subplots(2, 2, figsize=(12, 8))
    axs[0, 0].hist(original_input[:, 0], bins=50, alpha=0.7, color="orange")
    axs[0, 0].set_title('Original log10(T)')
    axs[0, 1].hist(normalized_input[:, 0], bins=50, alpha=0.7, color="darkorange")
    axs[0, 1].set_title('Normalized log10(T)')

    axs[1, 0].hist(original_input[:, 1], bins=50, alpha=0.7, color="steelblue")
    axs[1, 0].set_title('Original log10(nb)')
    axs[1, 1].hist(normalized_input[:, 1], bins=50, alpha=0.7, color='darkblue')
    axs[1, 1].set_title('Normalized log10(nb)')

    plt.tight_layout()
    plt.suptitle(f'Input Data: Original vs. Normalized ({dataset_name} Set)', y=1.02, fontsize=16)
    plt.savefig(os.path.join(save_path, f'normalization_input_{dataset_name.lower()}.png'), dpi=300, bbox_inches='tight')
    plt.show()

    # OUTPUTS
    fig, axs = plt.subplots(2, 2, figsize=(12, 8))
    axs[0, 0].hist(original_output[:, 0], bins=50, alpha=0.7, color="green")
    axs[0, 0].set_title('Original log10(P)')
    axs[0, 1].hist(normalized_output[:, 0], bins=50, alpha=0.7, color="darkgreen")
    axs[0, 1].set_title('Normalized log10(P)')

    axs[1, 0].hist(original_output[:, 1], bins=50, alpha=0.7, color="red")
    axs[1, 0].set_title('Original log10(S)')
    axs[1, 1].hist(normalized_output[:, 1], bins=50, alpha=0.7, color="darkred")
    axs[1, 1].set_title('Normalized log10(S)')

    plt.tight_layout()
    plt.suptitle(f'Output Data: Original vs. Normalized ({dataset_name} Set)', y=1.02, fontsize=16)
    plt.savefig(os.path.join(save_path, f'normalization_output_{dataset_name.lower()}.png'), dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# Visualizing the normalization for TRAINING
visualize_normalization(
    in_train, in_train_processed,
    out_train, out_train_processed,
    dataset_name="Train",
    save_path=plot_save_path
)

# Visualizing the normalization for VALIDATION
visualize_normalization(
    in_test, in_val_processed,
    out_test, out_val_processed,
    dataset_name="Validation",
    save_path=plot_save_path
)

visualize_normalization(
    in_test, in_test_processed,
    out_test, out_test,
    dataset_name="Test",
    save_path=plot_save_path
)

In [ ]:
# 3. Convert data to PyTorch tensors
in_train_tensor = torch.FloatTensor(in_train_processed)
out_train_tensor = torch.FloatTensor(out_train_processed)
in_test_tensor = torch.FloatTensor(in_test_processed)
out_test_tensor = torch.FloatTensor(out_test_processed)
in_val_tensor = torch.FloatTensor(in_val_processed)
out_val_tensor = torch.FloatTensor(out_val_processed)

It's clear that after the processing each data set is still representative of the EoS, keeping
- the characteristic peaks of q1 and q2 distributions
- long right tail in q1
- skewness in q2
- uniformity in T and nb

Moreover we see that the number of counts in each set corresponds to the amount of data that the represent from the total files. Stratification is not required